In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import dbuser, dbpassword
import datetime as dt

In [2]:
cab_rides = "Resources/cab_rides.csv"
cab_rides_df = pd.read_csv(cab_rides)
cab_rides_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [3]:
weather = "Resources/weather.csv"
weather_df = pd.read_csv(weather)
weather_df.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49


#### Convert epoch time to datetime

In [6]:
weather_df['DateTime']=pd.to_datetime(weather_df.time_stamp, unit='s', origin='unix')

weather_df.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind,DateTime
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25,2018-12-16 23:45:01
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32,2018-12-16 23:45:01
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07,2018-12-16 23:45:01
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09,2018-12-16 23:45:01
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49,2018-12-16 23:45:01


In [7]:
cab_rides_df['DateTime']=pd.to_datetime(cab_rides_df.time_stamp, unit='ms', origin='unix')

cab_rides_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,DateTime
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 09:30:07.890
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 02:00:23.677
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018-11-28 01:00:22.198
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018-11-30 04:53:02.749
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018-11-29 03:49:20.223


### Write function to bin timestamp by specified range
In this case binning is by Morning and Evening Rush-hour, Day Time and Night Time

In [8]:
def bin_f(x):
    if x.time() > dt.time(6) and x.time() < dt.time(10):
        return "6AM - 10AM"
    elif x.time() > dt.time(10) and x.time() < dt.time(15):
        return "10AM - 3PM"
    elif x.time() > dt.time(15) and x.time() < dt.time(19):
        return "3PM - 7PM"
    elif x.time() > dt.time(19) and x.time()< dt.time(22):
        return "7PM - 10PM"
    else:
        return "10PM - 6AM"
cab_rides_df["time_of_ride"] = cab_rides_df["DateTime"].apply(bin_f)
cab_rides_df['date'] = cab_rides_df.DateTime.dt.date

cab_rides_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,DateTime,time_of_ride,date
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 09:30:07.890,6AM - 10AM,2018-12-16
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 02:00:23.677,10PM - 6AM,2018-11-27
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018-11-28 01:00:22.198,10PM - 6AM,2018-11-28
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018-11-30 04:53:02.749,10PM - 6AM,2018-11-30
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018-11-29 03:49:20.223,10PM - 6AM,2018-11-29


In [9]:
weather_df['time_of_ride'] = weather_df['DateTime'].apply(bin_f)
weather_df['date'] = weather_df.DateTime.dt.date

weather_df.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind,DateTime,time_of_ride,date
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25,2018-12-16 23:45:01,10PM - 6AM,2018-12-16
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32,2018-12-16 23:45:01,10PM - 6AM,2018-12-16
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07,2018-12-16 23:45:01,10PM - 6AM,2018-12-16
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09,2018-12-16 23:45:01,10PM - 6AM,2018-12-16
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49,2018-12-16 23:45:01,10PM - 6AM,2018-12-16


### Aggregate surge pricing by ride provider

In [29]:
lyft_summary = cab_rides_df.loc[cab_rides_df['cab_type'] == 'Lyft'].groupby(['date', 'time_of_ride', 'destination'])['surge_multiplier'].mean().round(2).reset_index()
lyft_df = lyft_summary.rename(columns={'surge_multiplier': 'lyft_surge', 'destination':'location'})
lyft_df.head(20)

,date,time_of_ride,location,lyft_surge
0,2018-11-26,10AM - 3PM,Back Bay,1.04
1,2018-11-26,10AM - 3PM,Beacon Hill,1.02
2,2018-11-26,10AM - 3PM,Boston University,1.02
3,2018-11-26,10AM - 3PM,Fenway,1.02
4,2018-11-26,10AM - 3PM,Financial District,1.01
5,2018-11-26,10AM - 3PM,Haymarket Square,1.05
6,2018-11-26,10AM - 3PM,North End,1.04
7,2018-11-26,10AM - 3PM,North Station,1.03
8,2018-11-26,10AM - 3PM,Northeastern University,1.04
9,2018-11-26,10AM - 3PM,South Station,1.03


In [12]:
# rides_summary['Lyft Surge'] = cab_rides_df.loc[cab_rides_df['cab_type'] == 'Lyft'].groupby(['Date', 'Bin', 'destination'])['surge_multiplier'].mean().round(2).reset_index()
uber_summary = cab_rides_df.loc[cab_rides_df['cab_type'] == 'Uber'].groupby(['date', 'time_of_ride', 'destination'])['surge_multiplier'].mean().round(2).reset_index()

uber_df = uber_summary.rename(columns={'surge_multiplier': 'uber_surge', 'destination':'location'})
uber_df.head()

,date,time_of_ride,location,uber_surge
0,2018-11-26,10AM - 3PM,Back Bay,1.0
1,2018-11-26,10AM - 3PM,Beacon Hill,1.0
2,2018-11-26,10AM - 3PM,Boston University,1.0
3,2018-11-26,10AM - 3PM,Fenway,1.0
4,2018-11-26,10AM - 3PM,Financial District,1.0


In [13]:
ride_summary = pd.merge(lyft_df, uber_df, how="left", on=['date', 'time_of_ride', 'location'])
ride_summary

,date,time_of_ride,location,lyft_surge,uber_surge
0,2018-11-26,10AM - 3PM,Back Bay,1.04,1.0
1,2018-11-26,10AM - 3PM,Beacon Hill,1.02,1.0
2,2018-11-26,10AM - 3PM,Boston University,1.02,1.0
3,2018-11-26,10AM - 3PM,Fenway,1.02,1.0
4,2018-11-26,10AM - 3PM,Financial District,1.01,1.0
...,...,...,...,...,...
883,2018-12-18,7PM - 10PM,North Station,1.00,1.0
884,2018-12-18,7PM - 10PM,Northeastern University,1.02,1.0
885,2018-12-18,7PM - 10PM,South Station,1.00,1.0
886,2018-12-18,7PM - 10PM,Theatre District,1.05,1.0


### Aggregate weather by date, time and location

In [10]:
weather_group = weather_df.groupby(['date', 'time_of_ride', 'location'])

weather_summary = weather_group['temp', 'rain', 'wind'].mean().round(3).reset_index()

weather_summary

,date,time_of_ride,location,temp,rain,wind
0,2018-11-26,10AM - 3PM,Back Bay,41.182,0.008,2.720
1,2018-11-26,10AM - 3PM,Beacon Hill,41.151,0.004,2.746
2,2018-11-26,10AM - 3PM,Boston University,41.036,0.010,2.675
3,2018-11-26,10AM - 3PM,Fenway,41.053,0.016,2.654
4,2018-11-26,10AM - 3PM,Financial District,41.126,0.004,2.758
...,...,...,...,...,...,...
859,2018-12-18,6AM - 10AM,North Station,29.807,NaN,14.880
860,2018-12-18,6AM - 10AM,Northeastern University,29.722,NaN,14.690
861,2018-12-18,6AM - 10AM,South Station,30.168,NaN,14.935
862,2018-12-18,6AM - 10AM,Theatre District,29.718,NaN,14.878


### Summarize the dataset for further analysis

In [22]:
summary = pd.merge(ride_summary, weather_summary, how="left", on=['date', 'time_of_ride', 'location'])

summary = summary.rename(columns={'temp': 'avg_temp', 'rain':'avg_rain', 'wind':'avg_wind'})

summary.head()

,date,time_of_ride,location,lyft_surge,uber_surge,avg_temp,avg_rain,avg_wind
0,2018-11-26,10AM - 3PM,Back Bay,1.04,1.0,41.182,0.008,2.720
1,2018-11-26,10AM - 3PM,Beacon Hill,1.02,1.0,41.151,0.004,2.746
2,2018-11-26,10AM - 3PM,Boston University,1.02,1.0,41.036,0.010,2.675
3,2018-11-26,10AM - 3PM,Fenway,1.02,1.0,41.053,0.016,2.654
4,2018-11-26,10AM - 3PM,Financial District,1.01,1.0,41.126,0.004,2.758


### Create database connection

In [15]:
rds_connection_string = f"{dbuser}:{dbpassword}@localhost:5432/etl_project_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
# Confirm tables
engine.table_names()

['summary']

### Load DataFrames into database

In [ ]:
summary.to_sql(name='summary', con=engine, if_exists='append', index=False)

In [17]:
pd.read_sql_query('select * from summary', con=engine)

,date,time_of_ride,location,lyft_surge,uber_surge,avg_temp,avg_rain,avg_wind
0,2018-11-26,10AM - 3PM,Back Bay,1.04,1.0,41.182,0.008,2.720
1,2018-11-26,10AM - 3PM,Beacon Hill,1.02,1.0,41.151,0.004,2.746
2,2018-11-26,10AM - 3PM,Boston University,1.02,1.0,41.036,0.010,2.675
3,2018-11-26,10AM - 3PM,Fenway,1.02,1.0,41.053,0.016,2.654
4,2018-11-26,10AM - 3PM,Financial District,1.01,1.0,41.126,0.004,2.758
...,...,...,...,...,...,...,...,...
883,2018-12-18,7PM - 10PM,North Station,1.0,1.0,NaN,NaN,NaN
884,2018-12-18,7PM - 10PM,Northeastern University,1.02,1.0,NaN,NaN,NaN
885,2018-12-18,7PM - 10PM,South Station,1.0,1.0,NaN,NaN,NaN
886,2018-12-18,7PM - 10PM,Theatre District,1.05,1.0,NaN,NaN,NaN
